In [ ]:
library(dplyr)
library(factoextra)

df_pilot1_2022 <- readRDS(file = "../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/02_pilot1_2022.rds")

In [ ]:
df_pilot1_2022_clust <- df_pilot1_2022 %>% select(-id)
df_scaled <- scale(df_pilot1_2022_clust)

# Récupérer les moyennes et écarts-types
origines <- attr(df_scaled, "scaled:center")  # Moyennes utilisées pour centrer
ecarts_types <- attr(df_scaled, "scaled:scale")  # Écarts-types utilisés pour réduire

# Détermination du nombre optimal de clusters
# Méthode de l’élbow (coude) et l’indice de silhouette.

# Méthode du coude
fviz_nbclust(df_scaled, kmeans, method = "wss") +
  ggtitle("Méthode du coude")

# Indice de silhouette
fviz_nbclust(df_scaled, kmeans, method = "silhouette") +
  ggtitle("Indice de silhouette")

In [3]:
# Application de K-Means Clustering

# Choisissez un nombre de clusters approprié (par exemple, 3).
k <- 6

# K-Means Clustering
set.seed(123)  # Pour la reproductibilité
kmeans_result <- kmeans(df_scaled, centers = k, nstart = 25)

In [4]:
# Ajouter les clusters au dataframe original
df_pilot1_2022$cluster <- as.factor(kmeans_result$cluster)
df_pilot1_2022_clust$cluster <- as.factor(kmeans_result$cluster)

In [ ]:
kmeans_result

In [ ]:
# Visualisation des clusters

# Visualisation des clusters dans l’espace des deux premières dimensions :

# Utilisation de factoextra pour représenter les clusters
fviz_cluster(kmeans_result, data = df_scaled, ellipse.type = "euclid") +
  ggtitle("Visualisation des clusters avec K-Means")

In [7]:
# Effectuewr les moyennes de chaque groupe pour déterminert s'il y a des tendances marquantes
clusters_means <- aggregate(. ~ cluster, data = df_pilot1_2022_clust, function(x) mean(as.numeric(as.character(x)), na.rm = TRUE))

In [8]:
library(dplyr)
library(tidyr)

# Donner des noms aux clusters
df_pilot1_2022 <- df_pilot1_2022 %>%
  mutate(cluster_name = case_when(
    cluster == 1 ~ "Marie",
    cluster == 2 ~ "Jonathan",
    cluster == 3 ~ "Amélie",
    cluster == 4 ~ "Sam",
    cluster == 5 ~ "Connor",
    cluster == 6 ~ "Dominic"
  ))

# Créer des variables dummy
for (i in 1:6) {
  df_pilot1_2022[[paste0("cluster_", i)]] <- ifelse(df_pilot1_2022$cluster == i, 1, 0)
}


In [9]:
# Sauvegarder les résultats du clustering
saveRDS(origines, file="../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_scale_origins.rds")
saveRDS(ecarts_types, file="../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_scale_ecarts_types.rds")
saveRDS(kmeans_result, file = "../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_kmeans.rds")
saveRDS(df_pilot1_2022, file = "../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_2022.rds")
write.csv(clusters_means, file = "../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_clusters_desc.csv", row.names = FALSE)

In [ ]:
head(df_pilot1_2022)